In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, , , Waiting, , Waiting)

In [ ]:
df = spark.read.option("multiline", "true").json("Files/bing-latest-news.json")
# df now is a Spark DataFrame containing JSON data from "Files/bing-latest-news.json".
display(df)

StatementMeta(, , , Waiting, , Waiting)

SynapseWidget(Synapse.DataFrame, 82dbc55e-b004-49d8-b6ce-ef68cf79afca)

In [ ]:
from pyspark.sql.functions import explode

# 'value' sütununu explode ediyoruz
df_explode = df.select(explode(df["value"]).alias("json_object"))

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
display(df_explode)

StatementMeta(, , , Waiting, , Waiting)

SynapseWidget(Synapse.DataFrame, d73f1d87-d6f6-49b7-8122-e379c8144485)

In [ ]:
# JSON string olarak verileri topluyoruz
json_list = df_explode.toJSON().collect()

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
import json

# İlk JSON objesini işleyip kontrol ediyoruz
#news_json = json.loads(json_list[0])
#print(news_json['json_object']['description'])

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
# Verileri depolamak için listeler
title = []
description = []
category = []
url = []
image = []
provider = []
datePublished = []

# # Tüm kategori ve resim barındıran bilgileri topluyoruz
for json_str in json_list:
    try:
        article = json.loads(json_str)
        # Kategori ve resim varsa bilgileri listelere ekleyin
       # if article["json_object"].get("category") and article["json_object"].get("image", {}).get("thumbnail", {}).get("contentUrl"):
        title.append(article["json_object"]["name"])
        description.append(article["json_object"]["description"])  # Burada düzeltildi
        category.append(article["json_object"].get("category", "No Category"))
        url.append(article["json_object"]["url"])
        image.append(article["json_object"]["image"]["thumbnail"]["contentUrl"])
        provider.append(article["json_object"]["provider"][0]['name'])
        datePublished.append(article["json_object"]["datePublished"])

    except Exception as e:
        print(f"Error processing JSON object: {e}")
# Tüm bilgilerin hepsi
# for json_str in json_list:
#     try:
#         article = json.loads(json_str)
#         # Kategori ve resim bilgisi yoksa da haberleri işleyin
#         title.append(article["json_object"].get("name", "No Title"))
#         description.append(article["json_object"].get("description", "No Description"))  # Burada düzeltildi
#         category.append(article["json_object"].get("category", "No Category"))
#         url.append(article["json_object"].get("url", "No URL"))
#         image.append(article["json_object"].get("image", {}).get("thumbnail", {}).get("contentUrl", "No Image"))
#         provider.append(article["json_object"].get("provider", [{}])[0].get('name', "No Provider"))
#         datePublished.append(article["json_object"].get("datePublished", "No Date"))

#     except Exception as e:
#         print(f"Error processing JSON object: {e}")

StatementMeta(, , , Waiting, , Waiting)

Error processing JSON object: 'image'
Error processing JSON object: 'image'
Error processing JSON object: 'image'
Error processing JSON object: 'image'
Error processing JSON object: 'image'
Error processing JSON object: 'image'
Error processing JSON object: 'image'


In [ ]:
from pyspark.sql.types import StructType, StructField, StringType

data = list(zip(title,description,category,url,image,provider,datePublished))

schema = StructType([
    StructField("title",StringType(),True),
    StructField("description",StringType(),True),
    StructField("category",StringType(),True),
    StructField("url",StringType(),True),
    StructField("image",StringType(),True),
    StructField("provider",StringType(),True),
    StructField("datePublished",StringType(),True)
])

df_cleaned = spark.createDataFrame(data, schema=schema)


StatementMeta(, , , Waiting, , Waiting)

In [ ]:
display(df_cleaned)

StatementMeta(, , , Waiting, , Waiting)

SynapseWidget(Synapse.DataFrame, 42f4f56f-b04c-4d82-82cc-96f3c9e8de99)

In [ ]:
from pyspark.sql.functions import to_date, date_format

df_cleaned_final = df_cleaned.withColumn("datePublished",date_format(to_date("datePublished"),"dd-MMM-yyyy"))

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
display(df_cleaned_final)

StatementMeta(, , , Waiting, , Waiting)

SynapseWidget(Synapse.DataFrame, 4ec19d90-3650-4de7-af4e-a6735c14c947)

In [ ]:
#df_cleaned_final.write.format("delta").mode('overwrite').saveAsTable("bing_lake_db.tbl_latest_news")

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_lake_db.tbl_latest_news'

    df_cleaned_final.write.format("delta").saveAsTable("bing_lake_db.tbl_latest_news")

except AnalysisException:

    print("Table alredy Exists")

    df_cleaned_final.createOrReplaceTempView("vw_df_cleaned_final")

    spark.sql(f"""
    MERGE INTO {table_name} target_table
    USING vw_df_cleaned_final source_view
    ON source_view.url = target_table.url
    WHEN MATCHED AND 
        (source_view.title <> target_table.title OR
        source_view.description <> target_table.description OR
        source_view.category <> target_table.category OR
        source_view.image <> target_table.image OR
        source_view.provider <> target_table.provider OR
        source_view.datePublished <> target_table.datePublished)
    THEN 
        UPDATE SET 
        target_table.title = source_view.title,
        target_table.description = source_view.description,
        target_table.category = source_view.category,
        target_table.image = source_view.image,
        target_table.provider = source_view.provider,
        target_table.datePublished = source_view.datePublished
    WHEN NOT MATCHED THEN 
        INSERT (title, description, category, url, image, provider, datePublished)
        VALUES (source_view.title, source_view.description, source_view.category, 
                source_view.url, source_view.image, source_view.provider, 
                source_view.datePublished)
    """)


StatementMeta(, , , Waiting, , Waiting)

Table alredy Exists


In [ ]:
# from pyspark.sql.utils import AnalysisException

# try:
#     table_name = 'bing_lake_db.tbl_latest_news'

#     df_cleaned_final.write.format("delta").saveAsTable(table_name)

# except AnalysisException:
    
#     print("Table Already Exists")

#     df_cleaned_final.createOrReplaceTempView("vw_df_cleaned_final")

#     spark.sql(f"""
#         MERGE INTO {table_name} target_table
#         USING vw_df_cleaned_final source_view
#         ON source_view.url = target_table.url
#         WHEN MATCHED AND
#             source_view.title <> target_table.title OR
#             source_view.description <> target_table.description OR
#             source_view.category <> target_table.category OR
#             source_view.image <> target_table.image OR
#             source_view.provider <> target_table.provider OR
#             source_view.datePublished <> target_table.datePublished
#         THEN UPDATE SET *
#     """)


StatementMeta(, , , Waiting, , Waiting)

In [ ]:
display(df_cleaned_final)

StatementMeta(, , , Waiting, , Waiting)

SynapseWidget(Synapse.DataFrame, 72ffda55-358b-4eed-a2fe-d61a91c1a651)